### Env checking

In [2]:
import torch
print(torch.__version__)                   # In ra version, ví dụ 2.5.1
print(torch.cuda.is_available())           # True nếu GPU dùng được
print(torch.cuda.get_device_name(0))       # Tên GPU nếu có


2.5.1
True
NVIDIA GeForce RTX 4050 Laptop GPU


In [3]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available:", len(gpus))
for gpu in gpus:
    print("GPU Name:", gpu.name)


TensorFlow version: 2.10.0
Num GPUs Available: 1
GPU Name: /physical_device:GPU:0


### YOLO Detecting

In [7]:
from ultralytics import YOLO
import cv2
import os
from typing import Union, List

def detect_license_plates(image_paths: Union[str, List[str]],
                          model_path,
                          save_dir=r"D:\ViTraLP\output_yolo_crop_test",
                          conf_thresh=0.4):
    os.makedirs(save_dir, exist_ok=True)

    # Load YOLO model
    model = YOLO(model_path)

    # Nếu là 1 folder, lấy toàn bộ ảnh trong folder
    if isinstance(image_paths, str) and os.path.isdir(image_paths):
        supported_exts = ('.jpg', '.jpeg', '.png', '.bmp')
        image_paths = [os.path.join(image_paths, f) for f in os.listdir(image_paths)
                       if f.lower().endswith(supported_exts)]
    elif isinstance(image_paths, str):  # Là 1 file ảnh
        image_paths = [image_paths]

    all_cropped_paths = []

    for image_path in image_paths:
        results = model(image_path, conf=conf_thresh)
        result = results[0]
        boxes = result.boxes.xyxy.cpu().numpy()

        image = cv2.imread(image_path)
        cropped_paths = []

        for i, (x1, y1, x2, y2) in enumerate(boxes):
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            crop = image[y1:y2, x1:x2]

            basename = os.path.splitext(os.path.basename(image_path))[0]
            crop_filename = f"{basename}_plate_{i}.jpg"
            crop_path = os.path.join(save_dir, crop_filename)

            cv2.imwrite(crop_path, crop)
            cropped_paths.append(crop_path)

        all_cropped_paths.append({"image": image_path, "crops": cropped_paths})

    return all_cropped_paths


if __name__ == "__main__":
    # Bạn có thể truyền vào 1 folder chứa ảnh
    image_folder = r"C:\Users\ADMIN\Desktop\test_real\results\heavy"
    model_path = r"D:\ViTraLP\yolo_finetuned_weights\yolo11_medium_rainy_200_best.pt"
    
    results = detect_license_plates(image_folder, model_path)

    for result in results:
        print(f"Cropped plates for {result['image']}:")
        for path in result['crops']:
            print(" -", path)



image 1/1 C:\Users\ADMIN\Desktop\test_real\results\heavy\Dieu_0269_png.rf.ae3a7d4751b1ab6bd2efded1dd2fcf32.jpg: 640x640 1 LP, 6.5ms
Speed: 2.0ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\ADMIN\Desktop\test_real\results\heavy\Dieu_0269_png.rf.bb70c387d8e9f8c951a2447f63c4c894.jpg: 640x640 1 LP, 5.8ms
Speed: 2.0ms preprocess, 5.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\ADMIN\Desktop\test_real\results\heavy\Dieu_0269_png.rf.e9f24a1e2e608c2cb3471d5a47be62ab.jpg: 640x640 1 LP, 7.1ms
Speed: 2.7ms preprocess, 7.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\ADMIN\Desktop\test_real\results\heavy\Dieu_0277_png.rf.1e63158cf701baedf83e2f4bda73318b.jpg: 640x640 1 LP, 5.1ms
Speed: 1.8ms preprocess, 5.1ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\ADMIN\Desktop\test_real\results\heavy\Dieu_0277_png.rf.2d7438662

### PReNet enhance

In [ ]:
import os
import glob
import time
import cv2
import numpy as np
import torch
from torch.autograd import Variable
from utils import *  # Đảm bảo file utils.py nằm cùng thư mục hoặc trong PYTHONPATH
from networks import *  # Tương tự cho networks.py

# Cấu hình đường dẫn và thông số
logdir = r"D:\DSP_Project\derain_PReNet\PReNet\logs\finetuning\heavy\net_epoch50_heavy.pth"
data_path = r"C:\Users\ADMIN\Desktop\test_real\rainy\heavy"
save_path = r"C:\Users\ADMIN\Desktop\test_real\results\heavy"
use_GPU = True
gpu_id = "0"
recurrent_iter = 6

# Thiết lập GPU
if use_GPU:
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id

os.makedirs(save_path, exist_ok=True)

# Khởi tạo mô hình
print('Loading model...\n')
model = PReNet(recurrent_iter, use_GPU)
print_network(model)

if use_GPU:
    model = model.cuda()

# Load trọng số
model.load_state_dict(torch.load(logdir))
model.eval()

time_test = 0
count = 0

for img_name in os.listdir(data_path):
    if is_image(img_name):
        img_path = os.path.join(data_path, img_name)

        # Đọc và chuẩn hóa ảnh
        y = cv2.imread(img_path)
        b, g, r = cv2.split(y)
        y = cv2.merge([r, g, b])  # Chuyển sang RGB

        y = normalize(np.float32(y))
        y = np.expand_dims(y.transpose(2, 0, 1), 0)
        y = Variable(torch.Tensor(y))

        if use_GPU:
            y = y.cuda()

        with torch.no_grad():
            if use_GPU:
                torch.cuda.synchronize()
            start_time = time.time()

            out, _ = model(y)
            out = torch.clamp(out, 0., 1.)

            if use_GPU:
                torch.cuda.synchronize()
            end_time = time.time()

            dur_time = end_time - start_time
            time_test += dur_time
            print(img_name, ':', dur_time)

        # Chuyển tensor về ảnh
        out_np = out.data.cpu().numpy().squeeze() if use_GPU else out.data.numpy().squeeze()
        save_out = np.uint8(255 * out_np).transpose(1, 2, 0)
        b, g, r = cv2.split(save_out)
        save_out = cv2.merge([r, g, b])  # Trở lại BGR để lưu bằng OpenCV

        cv2.imwrite(os.path.join(save_path, img_name), save_out)
        count += 1

print('Avg. time:', time_test / count)


Loading model...

PReNet(
  (conv0): Sequential(
    (0): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (res_conv1): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (res_conv2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (res_conv3): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (res_conv4): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (re

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23348\1937457616.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(logdir))


Dieu_0269_png.rf.ae3a7d4751b1ab6bd2efded1dd2fcf32.jpg : 0.5168132781982422
Dieu_0269_png.rf.bb70c387d8e9f8c951a2447f63c4c894.jpg : 0.31981897354125977
Dieu_0269_png.rf.e9f24a1e2e608c2cb3471d5a47be62ab.jpg : 0.31943535804748535
Dieu_0277_png.rf.1e63158cf701baedf83e2f4bda73318b.jpg : 0.32265424728393555
Dieu_0277_png.rf.2d7438662ce4848d6f19bce0eb158043.jpg : 0.31897544860839844
Dieu_0277_png.rf.e1eceac15c516d34668a829641b62911.jpg : 0.32131004333496094
Dieu_0297_png.rf.2e02ec770094214aebe84fb37f4af7b8.jpg : 0.3188357353210449
Dieu_0297_png.rf.cba8d93ea7a38d0fc0cbe3a5046dd4a4.jpg : 0.3201022148132324
Dieu_0297_png.rf.d3702cc1d5eac01c05cb3e46b194cb47.jpg : 0.3212587833404541
Avg. time: 0.3421337869432237


### OCR recognizing

In [7]:
import os
from paddleocr import PaddleOCR

# Ép PaddleOCR chỉ dùng CPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Khởi tạo OCR với model nhận dạng tùy chỉnh
ocr_model = PaddleOCR(                                      
    use_angle_cls=False,
    det=False,
    rec=True,
    text_recognition_model_dir=r"D:/ViTraLP/original_weights_paddle",
    lang='en',
    show_log=False
)

# Sửa 2 ký tự đầu tiên nếu nhận nhầm
def correct_first_two_digits(text):
    char_to_digit = {
        'O': '0', 'o': '0', 'Q': '0',
        'I': '1', 'l': '1', 'L': '1',
        'Z': '2', 'S': '5', 'B': '8', 'G': '6'
    }
    text = text.strip()
    if len(text) < 2:
        return text
    new_text = list(text)
    for i in range(2):
        if not new_text[i].isdigit():
            new_text[i] = char_to_digit.get(new_text[i], '0')
    return ''.join(new_text)

# Hàm OCR một ảnh và in ra từng kết quả
def recognize_and_print(img_path):
    result = ocr_model.ocr(img_path, cls=False)
    print(f"\n=== {os.path.basename(img_path)} ===")
    if result and result[0]:
        for line in result[0]:
            (box, (text, confidence)) = line
            corrected_text = correct_first_two_digits(text)
            print(f"Text: {corrected_text}, Confidence: {confidence:.2f}")
    else:
        print("No text detected")

# OCR toàn bộ ảnh trong thư mục
image_folder = r"D:\ViTraLP\output_yolo_crop_test"
valid_exts = ['.jpg', '.jpeg', '.png', '.bmp']

for filename in os.listdir(image_folder):
    ext = os.path.splitext(filename)[-1].lower()
    if ext in valid_exts:
        img_path = os.path.join(image_folder, filename)
        try:
            recognize_and_print(img_path)
        except Exception as e:
            print(f"{filename}: Error - {str(e)}")



=== eh1_plate_0.jpg ===
Text: 29A, Confidence: 0.89
Text: 735.70, Confidence: 0.99

=== Tgmt_0641_png.rf.a483413bd4b4a3e16a2e6a28ab88948f_plate_0.jpg ===
Text: 29A, Confidence: 0.81
Text: 735.70, Confidence: 0.99


In [7]:
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=False)

# In toàn bộ thuộc tính để tìm path chứa model
print(dir(ocr))

[2025/07/21 10:23:50] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\ADMIN/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\ADMIN/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_leng